In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
import wave
import pyroomacoustics as pa
import glob
import random

from tqdm import tqdm

# 乱数を初期化
random.seed(0)

In [5]:
#　音声データをロードし、指定された秒数とサンプリングレートでリサンプル
def load_audio_file(file_path, length, sampling_rate=16000):
    data, sr = librosa.load(file_path, sr=sampling_rate)
    # data, sr = sf.read(file_path)
    # データが設定値よりも大きい場合は大きさを超えた分をカットする
    # データが設定値よりも小さい場合はデータの後ろを0でパディングする
    if len(data) > sampling_rate*length:
        data = data[:sampling_rate*length]
    else:
        data = np.pad(data, (0, max(0, sampling_rate*length - len(data))), "constant")
    return data

# 音声データを指定したサンプリングレートで保存
def save_audio_file(file_path, data, sampling_rate=16000):
    # librosa.output.write_wav(file_path, data, sampling_rate) # 正常に動作しないので変更
    sf.write(file_path, data, sampling_rate)

# 2つのオーディオデータを足し合わせる
def audio_mixer(data1, data2):
    assert len(data1) == len(data2)
    mixed_audio = data1 + data2
    return mixed_audio

# 音声データをスペクトログラムに変換する
def wave_to_spec(data, fft_size, hop_length):
    # 短時間フーリエ変換(STFT)を行い、スペクトログラムを取得
    spec = librosa.stft(data, n_fft=fft_size, hop_length=hop_length)
    mag = np.abs(spec) # 振幅スペクトログラムを取得
    phase = np.exp(1.j * np.angle(spec)) # 位相スペクトログラムを取得(フェーザ表示)
    # mel_spec = librosa.feature.melspectrogram(data, sr=sr, n_mels=128) # メルスペクトログラムを用いる場合はこっちを使う
    return mag, phase


# 音声に室内インパルス応答（Room Impulse Response）を畳み込む
def rir_convolve(wave_file, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR):
    """
    wave_file: 入力音声のパス
    sampling_rate: サンプリング周波数[Hz]
    doas: 音源の到来方向
    distance_mic_to_source: 音源とマイクロホンの距離 [m]
    mic_array_loc: マイクロホンアレイの位置座標
    R: 各マイクロホンの空間的な座標
    room_dim: 部屋の３次元形状を表す（単位はm）
    max_order: 部屋の壁で何回音が反射するか（反射しない場合0）
    absorption: 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    SNR: 音声と雑音の比率 [dB]
    """
    wave_files = [wave_file] # 後のためにリストに格納
    n_sources = len(wave_files)
#     print("音源数:", n_sources)
    source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
    """source_locations: (xyz, num_sources)"""
    source_locations[0,  :] = np.cos(doas[:, 1]) * np.cos(doas[:, 0]) # x = rcosφcosθ
    source_locations[1,  :] = np.sin(doas[:, 1]) * np.cos(doas[:, 0]) # y = rsinφcosθ
    source_locations[2,  :] = np.sin(doas[:, 0]) # z = rsinθ
    source_locations *= distance_mic_to_source
    source_locations += mic_array_loc[:, None] # マイクロホンアレイからの相対位置→絶対位置
    for i in range(n_sources):
        x = source_locations[0, i]
        y = source_locations[1, i]
        z = source_locations[2, i]
#         print("{}個目の音源の位置： (x, y, z) = ({}, {}, {})".format(i+1, x, y, z))
    # 音声波形の長さを調べる
    n_samples = 0
    # ファイルを読み込む
    for wave_file in wave_files:
        wav = wave.open(wave_file)
        if n_samples<wav.getnframes():
            n_samples=wav.getnframes()
        wav.close()
    clean_data = np.zeros([n_sources, n_samples])
    # ファイルを読み込む
    s = 0
    for wave_file in wave_files:
        wav = wave.open(wave_file)
        data = wav.readframes(wav.getnframes())
        data = np.frombuffer(data, dtype=np.int16)
        data = data/np.iinfo(np.int16).max
        clean_data[s, :wav.getnframes()] = data
        wav.close()
        s = s+1
    # 部屋を生成する
    room = pa.ShoeBox(room_dim, fs=sampling_rate, max_order=max_order, absorption=absorption)
    #　用いるマイクロホンアレイの情報を設定する
    room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))
    # 各音源をシミュレーションに追加する
    for s in range(n_sources):
        clean_data[s] /= np.std(clean_data[s])
        room.add_source(source_locations[:, s], signal=clean_data[s])
    # RIRのシミュレーション生成と音源信号への畳み込みを実行
    room.simulate(snr=SNR)
    convolved_wave = room.mic_array.signals.T/np.max(room.mic_array.signals.T)
#     print("畳み込み後の音声波形:", convolved_wave.shape)
    # インパルス応答の取得と残響時間（RT60）の取得
    impulse_responses = room.rir
    rt60 = pa.experimental.measure_rt60(impulse_responses[0][0], fs=sampling_rate)
#     print("残響時間:{} [sec]".format(rt60))
    
    return convolved_wave

In [6]:
if __name__ == '__main__':
    # 各パラメータを設定
    sampling_rate = 16000 # 作成するオーディオファイルのサンプリング周波数を指定
    audio_length = 3 # 単位は秒(second) → fft_size=1024,hop_length=768のとき、audio_length=6が最適化かも？
    train_val_ratio = 0.9 # trainデータとvalidationデータの割合
    fft_size = 512 # 高速フーリエ変換のフレームサイズ
    hop_length = 160 # 高速フーリエ変換においてフレームをスライドさせる幅
    
    # RIR生成用のパラメータ
    # 畳み込みに用いる波形
    # clean_wave_files = ["../data/NoisySpeechDatabase/noisy_trainset_28spk_wav_16kHz/p230_013.wav"]
    # 音声と雑音の比率 [dB]
    SNR = None
    # 音源とマイクロホンの距離 [m]
    distance_mic_to_source=2. 
    # 音源方向（音源が複数ある場合はリストに追加）
    azimuth = [0] # 方位角
    elevation = [np.pi/6] # 仰角
    # 部屋（シミュレーション環境）の設定
    room_width = 5.0
    room_length = 5.0
    room_height = 5.0
    # 部屋の残響を設定
    max_order = 0 #　部屋の壁で何回音が反射するか（反射しない場合0）
    absorption = None # 部屋の壁でどの程度音が吸収されるか （吸収されない場合None）
    # 以下は固定
    # 部屋の３次元形状を表す（単位はm）
    room_dim = np.r_[room_width, room_length, room_height]
    print("部屋の3次元形状：", room_dim)
    # マイクロホンアレイの中心位置
    nakbot_height = 0.57 # Nakbotの全長
    mic_array_height = nakbot_height - 0.04 # 0.04はTAMAGO-03マイクロホンアレイの頂上部からマイクロホンアレイ中心までの距離
    mic_array_loc = np.r_[room_width/2, room_length/2, 0] + [0, 0, mic_array_height] # 部屋の中央に配置されたNakbot上のマイクロホンアレイ
    print("マイクロホンアレイ中心座標：", mic_array_loc)
    # TAMAGO-03のマイクロホンアレイのマイクロホン配置（単位はm）
    mic_alignments = np.array(
    [
        [0.035, 0.0, 0.0],
        [0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [0.0, 0.035, 0.0],
        [-0.035/np.sqrt(2), 0.035/np.sqrt(2), 0.0],
        [-0.035, 0.0, 0.0],
        [-0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0],
        [0.0, -0.035, 0.0],
        [0.035/np.sqrt(2), -0.035/np.sqrt(2), 0.0]
    ])
    n_channels = np.shape(mic_alignments)[0]
    print("マイクロホン数：", n_channels)
    # get the microphone array　（各マイクロホンの空間的な座標）
    R = mic_alignments.T + mic_array_loc[:, None]
    """R: (3D coordinates [m], num_microphones)"""
    # 音源の位置（HARK座標系に対応） [仰角θ, 方位角φ]
    doas = np.array(
    [[elevation[0], azimuth[0]], # １個目の音源 
    # [elevation[1], azimuth[1]] # ２個目の音源
    ])
    
    # データセットを格納するディレクトリを作成
    save_dataset_dir = "../data/NoisySpeechDataset_for_unet_fft_512_8ch_1007/"
    os.makedirs(save_dataset_dir, exist_ok=True)

    # 学習・評価用
    # 人の発話音声のディレクトリを指定
    target_data_dir = "../data/NoisySpeechDatabase/clean_trainset_28spk_wav_16kHz/"
    # 外部雑音のディレクトリを指定
    interference_data_dir = "../data/NoisySpeechDatabase/interference_trainset_wav_16kHz/"
    # 混合音声のディレクトリを指定
    mixed_data_dir = "../data/NoisySpeechDatabase/noisy_trainset_28spk_wav_16kHz/"

    target_data_path_template = os.path.join(target_data_dir, "*.wav")
    target_list = glob.glob(target_data_path_template)
    # データセットをシャッフル
    random.shuffle(target_list)
    # データをtrainデータとvalidationデータに分割
    target_list_for_train = target_list[:int(len(target_list)*train_val_ratio)]
    target_list_for_train = random.sample(target_list_for_train, 3000) # データ量削減
    target_list_for_val = target_list[int(len(target_list)*train_val_ratio):]
    target_list_for_val = random.sample(target_list_for_val, 300) # データ量削減
    print("オリジナルデータの数:", len(target_list))
    print("trainデータの数:", len(target_list_for_train))
    print("validationデータの数:", len(target_list_for_val))
    
    # テスト用
    # 人の発話音声のディレクトリを指定
    target_data_dir_for_test= "../data/NoisySpeechDatabase/clean_testset_wav_16kHz/"
    # 外部雑音のディレクトリを指定
    interference_data_dir_for_test = "../data/NoisySpeechDatabase/interference_testset_wav_16kHz/"
    # 混合音声のディレクトリを指定
    mixed_data_dir_for_test = "../data/NoisySpeechDatabase/noisy_testset_wav_16kHz/"
    # テストデータのリストを作成
    target_data_path_template_for_test = os.path.join(target_data_dir_for_test, "*.wav")
    target_list_for_test = glob.glob(target_data_path_template_for_test)
    target_list_for_test = random.sample(target_list_for_test, 100) # データ量削減
    print("testデータの数:", len(target_list_for_test))
    
    # trainデータを作成
    print("trainデータ作成中")
    train_data_path = os.path.join(save_dataset_dir, "train")
    os.makedirs(train_data_path, exist_ok=True)
    for target_path in tqdm(target_list_for_train):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.npy" # (例)p226_001_target.npy
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_target_data = rir_convolve(target_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sampling_rate*audio_length, :]
        """convolved_target_data: (num_samples, num_channels=8)"""
        # マルチチャンネルオーディオデータをスペクトログラムに変換
        multichannel_target_spec_mag = []
        for i in range(convolved_target_data.shape[1]):
            # オーディオデータをスペクトログラムに変換
            convolved_target_mag, _ = wave_to_spec(convolved_target_data[:, i], fft_size, hop_length)
            multichannel_target_spec_mag.append(convolved_target_mag)
        multichannel_target_spec_mag = np.array(multichannel_target_spec_mag)
        """mulichannel_target_spec_mag: (num_channels=8, freq_bins=257, time_frames=301)"""
        max_mag = multichannel_target_spec_mag.max()
        normed_multichannel_target_spec_mag = multichannel_target_spec_mag / max_mag
        # .npy形式でスペクトログラムを保存
        target_save_path = os.path.join(train_data_path, target_file_name)
        np.save(target_save_path, normed_multichannel_target_spec_mag)
        # 混合音声の処理
        mixed_path = os.path.join(mixed_data_dir, file_num + ".wav")
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(mixed_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sampling_rate*audio_length, :]
        # オーディオデータをスペクトログラムに変換
        multichannel_mixed_spec_mag = []
        for i in range(convolved_mixed_data.shape[1]):
            # オーディオデータをスペクトログラムに変換
            convolved_mixed_mag, _ = wave_to_spec(convolved_mixed_data[:, i], fft_size, hop_length)
            multichannel_mixed_spec_mag.append(convolved_mixed_mag)
        multichannel_mixed_spec_mag = np.array(multichannel_mixed_spec_mag)
        """multichannel_mixed_spec_mag: (num_channels=8, freq_bins=257, time_frames=301)"""
        normed_multichannel_mixed_spec_mag = multichannel_mixed_spec_mag / max_mag
        # .npy形式でスペクトログラムを保存
        mixed_file_name = file_num + "_mixed.npy" # (例)p226_001_mixed.npy
        mixed_save_path = os.path.join(train_data_path, mixed_file_name)
        np.save(mixed_save_path, normed_multichannel_mixed_spec_mag)

    # validationデータを作成
    print("validationデータ作成中")
    val_data_path = os.path.join(save_dataset_dir, "val")
    os.makedirs(val_data_path, exist_ok=True)
    for target_path in tqdm(target_list_for_val):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.npy" # (例)p226_001_target.npy
         # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_target_data = rir_convolve(target_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sampling_rate*audio_length, :]
        # マルチチャンネルオーディオデータをスペクトログラムに変換
        multichannel_target_spec_mag = []
        for i in range(convolved_target_data.shape[1]):
            # オーディオデータをスペクトログラムに変換
            convolved_target_mag, _ = wave_to_spec(convolved_target_data[:, i], fft_size, hop_length)
            multichannel_target_spec_mag.append(convolved_target_mag)
        multichannel_target_spec_mag = np.array(multichannel_target_spec_mag)
        """mulichannel_target_spec_mag: (num_channels=8, freq_bins=257, time_frames=301)"""
        max_mag = multichannel_target_spec_mag.max()
        normed_multichannel_target_spec_mag = multichannel_target_spec_mag / max_mag
        # .npy形式でスペクトログラムを保存
        target_save_path = os.path.join(val_data_path, target_file_name)
        np.save(target_save_path, normed_multichannel_target_spec_mag)
        # 混合音声の処理
        mixed_path = os.path.join(mixed_data_dir, file_num + ".wav")
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(mixed_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sampling_rate*audio_length, :]
        # オーディオデータをスペクトログラムに変換
        multichannel_mixed_spec_mag = []
        for i in range(convolved_mixed_data.shape[1]):
            # オーディオデータをスペクトログラムに変換
            convolved_mixed_mag, _ = wave_to_spec(convolved_mixed_data[:, i], fft_size, hop_length)
            multichannel_mixed_spec_mag.append(convolved_mixed_mag)
        multichannel_mixed_spec_mag = np.array(multichannel_mixed_spec_mag)
        """multichannel_mixed_spec_mag: (num_channels=8, freq_bins=257, time_frames=301)"""
        normed_multichannel_mixed_spec_mag = multichannel_mixed_spec_mag / max_mag
        # .npy形式でスペクトログラムを保存
        mixed_file_name = file_num + "_mixed.npy" # (例)p226_001_mixed.npy
        mixed_save_path = os.path.join(val_data_path, mixed_file_name)
        np.save(mixed_save_path, normed_multichannel_mixed_spec_mag)
        
    # testデータを作成
    print("testデータ作成中")
    test_data_path = os.path.join(save_dataset_dir, "test")
    os.makedirs(test_data_path, exist_ok=True)
    for target_path in tqdm(target_list_for_test):
        file_num = os.path.basename(target_path).split('.')[0] # (例)p226_001
        target_file_name = file_num + "_target.wav" # (例)p226_001_target.wav
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_target_data = rir_convolve(target_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_target_data = convolved_target_data[:sampling_rate*audio_length, :]
        """convolved_target_data: (num_samples, num_channels=8)"""
        # 音声データのサンプリング周波数を指定して保存
        target_file_path = os.path.join(test_data_path, target_file_name)
        save_audio_file(target_file_path, convolved_target_data, sampling_rate)
        # 混合音声の処理
        mixed_path = os.path.join(mixed_data_dir_for_test, file_num + ".wav")
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_mixed_data = rir_convolve(mixed_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_mixed_data = convolved_mixed_data[:sampling_rate*audio_length, :]
        # 音声データのサンプリング周波数を指定して保存
        mixed_file_name = file_num + "_mixed.wav" # (例)p226_001_mixed.npy
        mixed_file_path = os.path.join(test_data_path, mixed_file_name)
        save_audio_file(mixed_file_path, convolved_mixed_data, sampling_rate)
         # 干渉音の処理
        interference_path = os.path.join(interference_data_dir_for_test, file_num + ".wav")
        # 音声にRIRを畳み込みながらマルチチャンネルに拡張
        convolved_interference_data = rir_convolve(interference_path, sampling_rate, doas, distance_mic_to_source, \
                 mic_array_loc, R, room_dim, max_order, absorption, SNR)
        # RIRの長さ-1サンプル分オーディオデータが長くなるので、元に戻す
        convolved_interference_data = convolved_interference_data[:sampling_rate*audio_length, :]
        # 音声データのサンプリング周波数を指定して保存
        interference_file_name = file_num + "_interference.wav" # (例)p226_001_interference.wav
        interference_file_path = os.path.join(test_data_path, interference_file_name)
        save_audio_file(interference_file_path, convolved_interference_data, sampling_rate)
            

    print("データ作成完了　保存先：{}".format(save_dataset_dir))


  0%|          | 0/3000 [00:00<?, ?it/s]

部屋の3次元形状： [5. 5. 5.]
マイクロホンアレイ中心座標： [2.5  2.5  0.53]
マイクロホン数： 8
オリジナルデータの数: 11572
trainデータの数: 3000
validationデータの数: 300
testデータの数: 100
trainデータ作成中


  0%|          | 0/300 [00:00<?, ?it/s]

validationデータ作成中


  0%|          | 0/100 [00:00<?, ?it/s]

testデータ作成中


100%|██████████| 100/100 [00:24<00:00,  4.00it/s]

データ作成完了　保存先：../data/NoisySpeechDataset_for_unet_fft_512_8ch_1007/
